<img src='http://imgur.com/1ZcRyrc.png' style='float: left; margin: 20px; height: 55px'>

# Capstone: A Lindy Hop Classifier

In [1]:
from collections import defaultdict

import cv2
import numpy as np
import pandas as pd

from ultralytics import YOLO
import os

import seaborn as sns
import matplotlib.pyplot as plt

## 1. Load Files

In [2]:
# custom function to iterate through a folder and generate df

def generate_df(folder_name):
    
    # define dictionary
    
    KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}
    
    # create columns to track the person and each recording analysed
    columns = ['person','recording_id']
    
    for key, value in KEYPOINT_DICT.items():
        columns.extend([key + '_x', key + '_y']) # Add columns capturing x and y for each keypoint to the df
    
    df = pd.DataFrame(columns=columns)
    
    # define folder source
    source = folder_name
    
    # get a list of video files in the source directory
    video_files = [os.path.join(source, file) for file in os.listdir(source) if file.endswith('.mov')]

    # run YOLO inference
    model = YOLO("YOLOv8m-pose.pt")
    
    # set count for recording ID
    file_number = 1

   # iterate through folder to use YOLO inference on stream, saving annotated video
    for file in video_files:
       
        # Store the track history
        track_history = defaultdict(lambda: [])

        results = model.track(file, conf=0.5, stream = True) #save = True

        # process through results generator
        for r in results:
            
            # get the boxes and track IDs of boxes
            boxes = r[0].boxes.xywh.cpu()
            track_ids = r[0].boxes.id.int().cpu().tolist()

            # plot the tracks
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 90 tracks for 90 frames
                    track.pop(0)

            # retrieve keypoints, add keypoints to df
            row = r.keypoints.xyn.cpu().numpy()[0].flatten().tolist()
            row.insert(0, track_id) # Insert placeholder for the person detected # But how to tell who the keypoints belong to?
            row.insert(1, file_number)  # Insert "1" at index 0 of each row to label the person
            df.loc[len(df)] = row # Append modified list (i.e. row) to the dataframe X

        # Increment the video file number for the next iteration
        file_number += 1

    return df

In [3]:
# Note that some video files were invalid when code was subsequently re-ran
# The CSV used for this project is the first one where all video files were accessible

open_path = "/Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open"
df_open = generate_df(open_path)


[mov,mp4,m4a,3gp,3g2,mj2 @ 0x169c08900] moov atom not found
OpenCV: Couldn't read video stream from file "/Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 3.mov"

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x1699189f0] moov atom not found
OpenCV: Couldn't read video stream from file "/Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/06 swingout 5.mov"

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x15d7fb0c0] moov atom not found
OpenCV: Couldn't read video stream from file "/Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 2.mov"

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x1699189f0] moov atom not found
OpenCV: Couldn't read video stream from file "/Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 17.mov"

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x168ff7a40] moov atom not found
OpenCV: Couldn't read video stream from file "/Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 rec

video 1/1 (37/125) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/98 swingout 7.mov: 384x640 1 person, 156.4ms
video 1/1 (38/125) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/98 swingout 7.mov: 384x640 2 persons, 157.9ms
video 1/1 (39/125) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/98 swingout 7.mov: 384x640 2 persons, 162.0ms
video 1/1 (40/125) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/98 swingout 7.mov: 384x640 2 persons, 158.3ms
video 1/1 (41/125) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/98 swingout 7.mov: 384x640 2 persons, 157.8ms
video 1/1 (42/125) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/98 swingout 7.mov: 384x640 2 persons, 159.3ms
video 1/1 (43/125) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/98 swingout 7.mov:

video 1/1 (93/125) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/98 swingout 7.mov: 384x640 2 persons, 157.2ms
video 1/1 (94/125) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/98 swingout 7.mov: 384x640 2 persons, 161.4ms
video 1/1 (95/125) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/98 swingout 7.mov: 384x640 2 persons, 164.3ms
video 1/1 (96/125) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/98 swingout 7.mov: 384x640 2 persons, 163.3ms
video 1/1 (97/125) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/98 swingout 7.mov: 384x640 2 persons, 158.2ms
video 1/1 (98/125) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/98 swingout 7.mov: 384x640 2 persons, 162.9ms
video 1/1 (99/125) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/98 swingout 7.mov

video 1/1 (34/123) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/05 swingout 2.mov: 384x640 1 person, 172.7ms
video 1/1 (35/123) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/05 swingout 2.mov: 384x640 1 person, 163.6ms
video 1/1 (36/123) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/05 swingout 2.mov: 384x640 1 person, 165.2ms
video 1/1 (37/123) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/05 swingout 2.mov: 384x640 1 person, 162.2ms
video 1/1 (38/123) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/05 swingout 2.mov: 384x640 1 person, 162.6ms
video 1/1 (39/123) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/05 swingout 2.mov: 384x640 1 person, 160.1ms
video 1/1 (40/123) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/05 swingout 2.mov: 384x

video 1/1 (90/123) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/05 swingout 2.mov: 384x640 2 persons, 164.7ms
video 1/1 (91/123) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/05 swingout 2.mov: 384x640 2 persons, 161.3ms
video 1/1 (92/123) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/05 swingout 2.mov: 384x640 2 persons, 159.2ms
video 1/1 (93/123) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/05 swingout 2.mov: 384x640 2 persons, 160.2ms
video 1/1 (94/123) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/05 swingout 2.mov: 384x640 2 persons, 160.8ms
video 1/1 (95/123) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/05 swingout 2.mov: 384x640 2 persons, 159.7ms
video 1/1 (96/123) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/05 swingout 2.mov

...

video 1/1 (14/102) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 26.mov: 384x640 2 persons, 199.7ms
video 1/1 (15/102) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 26.mov: 384x640 2 persons, 200.9ms
video 1/1 (16/102) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 26.mov: 384x640 2 persons, 200.0ms
video 1/1 (17/102) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 26.mov: 384x640 2 persons, 199.5ms
video 1/1 (18/102) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 26.mov: 384x640 2 persons, 202.9ms
video 1/1 (19/102) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 26.mov: 384x640 2 persons, 209.4ms
video 1/1 (20/102) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout

video 1/1 (70/102) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 26.mov: 384x640 1 person, 200.9ms
video 1/1 (71/102) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 26.mov: 384x640 1 person, 194.1ms
video 1/1 (72/102) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 26.mov: 384x640 1 person, 202.8ms
video 1/1 (73/102) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 26.mov: 384x640 1 person, 191.5ms
video 1/1 (74/102) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 26.mov: 384x640 1 person, 196.9ms
video 1/1 (75/102) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 26.mov: 384x640 2 persons, 200.5ms
video 1/1 (76/102) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 26.m

video 1/1 (29/111) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 9.mov: 384x640 2 persons, 196.4ms
video 1/1 (30/111) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 9.mov: 384x640 2 persons, 196.7ms
video 1/1 (31/111) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 9.mov: 384x640 2 persons, 197.4ms
video 1/1 (32/111) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 9.mov: 384x640 2 persons, 199.5ms
video 1/1 (33/111) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 9.mov: 384x640 1 person, 199.3ms
video 1/1 (34/111) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 9.mov: 384x640 1 person, 199.1ms
video 1/1 (35/111) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 9.mov: 

video 1/1 (85/111) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 9.mov: 384x640 2 persons, 201.5ms
video 1/1 (86/111) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 9.mov: 384x640 2 persons, 201.4ms
video 1/1 (87/111) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 9.mov: 384x640 2 persons, 188.6ms
video 1/1 (88/111) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 9.mov: 384x640 2 persons, 191.1ms
video 1/1 (89/111) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 9.mov: 384x640 2 persons, 191.8ms
video 1/1 (90/111) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 9.mov: 384x640 2 persons, 196.9ms
video 1/1 (91/111) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 9.mov

video 1/1 (46/114) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 8.mov: 384x640 2 persons, 196.6ms
video 1/1 (47/114) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 8.mov: 384x640 2 persons, 195.3ms
video 1/1 (48/114) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 8.mov: 384x640 2 persons, 196.9ms
video 1/1 (49/114) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 8.mov: 384x640 2 persons, 198.3ms
video 1/1 (50/114) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 8.mov: 384x640 2 persons, 194.8ms
video 1/1 (51/114) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 8.mov: 384x640 2 persons, 195.3ms
video 1/1 (52/114) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/open/08 swingout 8.mov

In [4]:
df_open.to_csv('swingout_open.csv',index=False)

In [5]:
df_open['person'].value_counts()

person
1.0    725
2.0    403
3.0    283
4.0     15
Name: count, dtype: int64

In [6]:
# Note that some video files were invalid when code was subsequently re-ran
# The CSV used for this project is the first one where all video files were accessible

closed_path = "/Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close"
df_close = generate_df(closed_path)


video 1/1 (1/117) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/08 swingout 1.mov: 384x640 2 persons, 191.5ms
video 1/1 (2/117) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/08 swingout 1.mov: 384x640 2 persons, 190.1ms
video 1/1 (3/117) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/08 swingout 1.mov: 384x640 2 persons, 190.3ms
video 1/1 (4/117) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/08 swingout 1.mov: 384x640 2 persons, 191.6ms
video 1/1 (5/117) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/08 swingout 1.mov: 384x640 2 persons, 191.9ms
video 1/1 (6/117) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/08 swingout 1.mov: 384x640 2 persons, 183.2ms
video 1/1 (7/117) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/08 swingout 1.mo

video 1/1 (57/117) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/08 swingout 1.mov: 384x640 2 persons, 194.8ms
video 1/1 (58/117) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/08 swingout 1.mov: 384x640 2 persons, 194.5ms
video 1/1 (59/117) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/08 swingout 1.mov: 384x640 2 persons, 192.1ms
video 1/1 (60/117) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/08 swingout 1.mov: 384x640 2 persons, 191.7ms
video 1/1 (61/117) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/08 swingout 1.mov: 384x640 2 persons, 189.2ms
video 1/1 (62/117) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/08 swingout 1.mov: 384x640 2 persons, 192.1ms
video 1/1 (63/117) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/08 swingou

video 1/1 (5/137) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/07 swingout 1.mov: 384x640 2 persons, 186.3ms
video 1/1 (6/137) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/07 swingout 1.mov: 384x640 2 persons, 190.4ms
video 1/1 (7/137) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/07 swingout 1.mov: 384x640 2 persons, 189.3ms
video 1/1 (8/137) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/07 swingout 1.mov: 384x640 2 persons, 188.9ms
video 1/1 (9/137) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/07 swingout 1.mov: 384x640 2 persons, 190.9ms
video 1/1 (10/137) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/07 swingout 1.mov: 384x640 2 persons, 189.7ms
video 1/1 (11/137) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/07 swingout 1.m

video 1/1 (61/137) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/07 swingout 1.mov: 384x640 2 persons, 188.5ms
video 1/1 (62/137) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/07 swingout 1.mov: 384x640 2 persons, 190.6ms
video 1/1 (63/137) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/07 swingout 1.mov: 384x640 2 persons, 190.2ms
video 1/1 (64/137) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/07 swingout 1.mov: 384x640 2 persons, 190.6ms
video 1/1 (65/137) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/07 swingout 1.mov: 384x640 2 persons, 191.9ms
video 1/1 (66/137) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/07 swingout 1.mov: 384x640 2 persons, 192.6ms
video 1/1 (67/137) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/07 swingou

video 1/1 (117/137) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/07 swingout 1.mov: 384x640 2 persons, 189.8ms
video 1/1 (118/137) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/07 swingout 1.mov: 384x640 2 persons, 190.6ms
video 1/1 (119/137) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/07 swingout 1.mov: 384x640 2 persons, 191.1ms
video 1/1 (120/137) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/07 swingout 1.mov: 384x640 2 persons, 187.5ms
video 1/1 (121/137) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/07 swingout 1.mov: 384x640 2 persons, 195.0ms
video 1/1 (122/137) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/07 swingout 1.mov: 384x640 2 persons, 190.9ms
video 1/1 (123/137) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/07 

...

video 1/1 (76/125) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/05 swingout 10.mov: 384x640 2 persons, 208.4ms
video 1/1 (77/125) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/05 swingout 10.mov: 384x640 2 persons, 207.3ms
video 1/1 (78/125) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/05 swingout 10.mov: 384x640 2 persons, 206.1ms
video 1/1 (79/125) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/05 swingout 10.mov: 384x640 2 persons, 207.6ms
video 1/1 (80/125) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/05 swingout 10.mov: 384x640 2 persons, 206.3ms
video 1/1 (81/125) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/05 swingout 10.mov: 384x640 2 persons, 208.0ms
video 1/1 (82/125) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/05 s

video 1/1 (12/113) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 1.mov: 384x640 2 persons, 201.4ms
video 1/1 (13/113) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 1.mov: 384x640 2 persons, 205.7ms
video 1/1 (14/113) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 1.mov: 384x640 2 persons, 207.3ms
video 1/1 (15/113) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 1.mov: 384x640 2 persons, 205.7ms
video 1/1 (16/113) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 1.mov: 384x640 2 persons, 202.1ms
video 1/1 (17/113) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 1.mov: 384x640 2 persons, 205.4ms
video 1/1 (18/113) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A 

video 1/1 (68/113) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 1.mov: 384x640 2 persons, 203.1ms
video 1/1 (69/113) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 1.mov: 384x640 2 persons, 208.5ms
video 1/1 (70/113) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 1.mov: 384x640 2 persons, 201.8ms
video 1/1 (71/113) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 1.mov: 384x640 2 persons, 202.2ms
video 1/1 (72/113) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 1.mov: 384x640 2 persons, 204.0ms
video 1/1 (73/113) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 1.mov: 384x640 2 persons, 202.9ms
video 1/1 (74/113) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A 

video 1/1 (29/97) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 5.mov: 384x640 2 persons, 197.4ms
video 1/1 (30/97) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 5.mov: 384x640 2 persons, 196.3ms
video 1/1 (31/97) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 5.mov: 384x640 2 persons, 196.7ms
video 1/1 (32/97) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 5.mov: 384x640 2 persons, 199.3ms
video 1/1 (33/97) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 5.mov: 384x640 2 persons, 197.4ms
video 1/1 (34/97) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 5.mov: 384x640 2 persons, 196.3ms
video 1/1 (35/97) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingou

video 1/1 (85/97) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 5.mov: 384x640 2 persons, 196.3ms
video 1/1 (86/97) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 5.mov: 384x640 2 persons, 197.0ms
video 1/1 (87/97) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 5.mov: 384x640 2 persons, 195.8ms
video 1/1 (88/97) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 5.mov: 384x640 2 persons, 200.8ms
video 1/1 (89/97) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 5.mov: 384x640 2 persons, 199.3ms
video 1/1 (90/97) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingout 5.mov: 384x640 2 persons, 197.0ms
video 1/1 (91/97) /Users/eugenia/Desktop/DSI-SG-39/Capstone Project/231023 recordings/swingout/close/00A swingou

In [9]:
df_close.to_csv('swingout_closed.csv',index=False)

In [10]:
df_close['person'].value_counts()

person
1.0    1580
2.0     852
3.0      28
Name: count, dtype: int64